In [271]:
import pandas as pd

# 모든 컬럼, 로우 출력/요약 출력
pd.set_option('display.max_columns', None)
pd.reset_option('display.max_rows', None)

### 총자산증가율, 자기자본증가율, 정상영업손익증가율, 순이익증가율, 매출액증가율 파생변수 만들기
#### [K-IFRS] 연결데이터와 별도데이터 병합하기

In [272]:
# 데이터 불러오기
con = pd.read_csv('데이터/data/연결노션성장성.csv', encoding='CP949')
sep = pd.read_csv('데이터/data/별도노션성장성.csv', encoding='CP949')


# 연결, 병합 데이터 비교하고 확인
print('<연결>')
print('행렬 수 : ', con.shape)
print('회사 수 : ', con['회사명'].nunique())

print('\n<별도>')
print('행렬 수 : ', sep.shape)
print('회사 수 : ', sep['회사명'].nunique())

<연결>
행렬 수 :  (20947, 10)
회사 수 :  2456

<별도>
행렬 수 :  (20947, 10)
회사 수 :  2456


In [273]:
# 데이터 프레임 결측치 합 확인
# con.isna().sum()
# sep.isna().sum()

In [274]:
# 컬럼명 확인
# con.columns
# sep.columns


# 컬럼명 불필요 문자 제거
con.columns = con.columns.str.replace('수익', '')
con.columns = con.columns.str.replace('(', '')
con.columns = con.columns.str.replace(')', '')
con.columns = con.columns.str.replace('IFRS연결', '')
con.columns = con.columns.str.replace('*', '')
con.columns = con.columns.str.replace('보고서기재', '')
con.columns = con.columns.str.replace('손실', '')
con.columns = con.columns.str.replace('정상', '')
con.columns = con.columns.str.replace(' ', '')
con.columns = con.columns.str.replace('천원', '')


sep.columns = sep.columns.str.replace('수익', '')
sep.columns = sep.columns.str.replace('(', '')
sep.columns = sep.columns.str.replace(')', '')
sep.columns = sep.columns.str.replace('IFRS', '')
sep.columns = sep.columns.str.replace('*', '')
sep.columns = sep.columns.str.replace('보고서기재', '')
sep.columns = sep.columns.str.replace('손실', '')
sep.columns = sep.columns.str.replace('정상', '')
sep.columns = sep.columns.str.replace(' ', '')
sep.columns = sep.columns.str.replace('천원', '')


# 칼럼 재배치
con = con[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '자산', '자본', '영업손익', '당기순이익', '매출액']]
sep = sep[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '자산', '자본', '영업손익', '당기순이익', '매출액']]

<ipython-input-274-d9cd6727260b>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  con.columns = con.columns.str.replace('(', '')
<ipython-input-274-d9cd6727260b>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  con.columns = con.columns.str.replace(')', '')
<ipython-input-274-d9cd6727260b>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  con.columns = con.columns.str.replace('*', '')
<ipython-input-274-d9cd6727260b>:20: FutureWarning: The default value of regex will change from True to False in a future version. In additio

#### 연결기업 결측치를 개별기업 데이터로 대체

In [275]:
consep = con.copy()

# 연결기업 데이터프레임의 결측치를 개별기업 데이터로 대체
for i in consep.columns:
    consep[i] = consep[i].fillna(sep[i])


# ts연결별도병합 데이터 프레임 저장
consep.to_csv('데이터/1차 전처리/ifrs성장성연결별도병합.csv', encoding='CP949', index=False)

In [276]:
# 연결, 별도, 병합 데이터 비교하고 확인
print('<연결>')
print('행렬 수 : ', con.shape)
print('회사 수 : ', con['회사명'].nunique())

print('\n<별도>')
print('행렬 수 : ', sep.shape)
print('회사 수 : ', sep['회사명'].nunique())

print('\n<병합>')
print('행렬 수 : ', consep.shape)
print('회사 수 : ', consep['회사명'].nunique())

<연결>
행렬 수 :  (20947, 10)
회사 수 :  2456

<별도>
행렬 수 :  (20947, 10)
회사 수 :  2456

<병합>
행렬 수 :  (20947, 10)
회사 수 :  2456


#### 파생변수 만들기
    컬럼 하나 새로 만들어서
    포문 공식 : {(당기 - 전기) / 전기} * 100 값을 채워넣으려고 함
    근데 만약 전기영업손익 < 0 면, 계산 => {(전기 - 당기) / 당기} * 100 로 채워넣을거임

In [277]:
consep.columns

Index(['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '자산', '자본', '영업손익', '당기순이익',
       '매출액'],
      dtype='object')

In [278]:
# 전기 데이터 만들기
consep['전기영업손익'] = consep['영업손익'].shift().where(consep.회사명.eq(consep.회사명.shift()))
consep['전기순이익'] = consep['당기순이익'].shift().where(consep.회사명.eq(consep.회사명.shift()))
consep['전기매출액'] = consep['매출액'].shift().where(consep.회사명.eq(consep.회사명.shift()))
consep['전기자산'] = consep['자산'].shift().where(consep.회사명.eq(consep.회사명.shift()))
consep['전기자본'] = consep['자본'].shift().where(consep.회사명.eq(consep.회사명.shift()))

consep2 = consep.copy()

# 컬럼명 재정렬
consep2 = consep2[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '자산', '전기자산', '자본', '전기자본', '영업손익', '전기영업손익', '당기순이익', '전기순이익', '매출액', '전기매출액']]


# 성장성 재무비율 구하기
consep2['총자산증가율'] = ((consep2['자산'] - consep2['전기자산'])/consep2['전기자산'])*100
consep2['자기자본증가율'] = ((consep2['자본'] - consep2['전기자본'])/abs(consep2['전기자본']))*100
for i, row in consep2.iterrows () :
    if consep2.at[i,'전기영업손익'] > 0 or consep2.at[i,'전기영업손익'] < 0 and consep2.at[i,'영업손익'] < 0:
        consep2.at[i,'정상영업손익증가율'] = ((consep2.at[i,'영업손익'] - consep2.at[i,'전기영업손익'])/consep2.at[i,'전기영업손익'])*100
    elif consep2.at[i,'전기영업손익'] < 0 and consep2.at[i,'영업손익'] > 0:
        consep2.at[i,'정상영업손익증가율'] = ((consep2.at[i,'전기영업손익'] - consep2.at[i,'영업손익'])/consep2.at[i,'영업손익'])*100

for i, row in consep2.iterrows () :
    if consep2.at[i,'전기순이익'] > 0 or consep2.at[i,'전기순이익'] < 0 and consep2.at[i,'당기순이익'] < 0:
        consep2.at[i,'순이익증가율'] = ((consep2.at[i,'당기순이익'] - consep2.at[i,'전기순이익'])/consep2.at[i,'전기순이익'])*100
    elif consep2.at[i,'전기순이익'] < 0 and consep2.at[i,'당기순이익'] > 0:
        consep2.at[i,'순이익증가율'] = ((consep2.at[i,'전기순이익'] - consep2.at[i,'당기순이익'])/consep2.at[i,'당기순이익'])*100

for i, row in consep2.iterrows () :
    if consep2.at[i,'전기매출액'] > 0 or consep2.at[i,'전기매출액'] < 0 and consep2.at[i,'매출액'] < 0:
        consep2.at[i,'매출액증가율'] = ((consep2.at[i,'매출액'] - consep2.at[i,'전기매출액'])/consep2.at[i,'전기매출액'])*100
    elif consep2.at[i,'전기매출액'] < 0 and consep2.at[i,'매출액'] > 0:
        consep2.at[i,'매출액증가율'] = ((consep2.at[i,'전기매출액'] - consep2.at[i,'매출액'])/consep2.at[i,'매출액'])*100


# 데이터 프레임 저장
consep2.to_csv('데이터/1차 전처리/ifrs성장성파생변수.csv', encoding='CP949', index=False)

--------

### 총자산증가율, 자기자본증가율, 정상영업손익증가율, 순이익증가율, 매출액증가율 파생변수 만들기
#### [K-GAAP] 연결데이터와 별도데이터 병합하기

In [279]:
# 데이터 불러오기
kgcon = pd.read_csv('데이터/data/케이갭연결성장성.csv', encoding='CP949')
kgsep = pd.read_csv('데이터/data/케이갭별도성장성.csv', encoding='CP949')

# 연결, 병합 데이터 비교하고 확인
print('<연결>')
print('행렬 수 : ', kgcon.shape)
print('회사 수 : ', kgcon['회사명'].nunique())

print('\n<별도>')
print('행렬 수 : ', kgsep.shape)
print('회사 수 : ', kgsep['회사명'].nunique())

<연결>
행렬 수 :  (20831, 10)
회사 수 :  2388

<별도>
행렬 수 :  (20831, 10)
회사 수 :  2388


In [280]:
# 데이터 프레임 결측치 합 확인
# kgcon.isna().sum()
# kgsep.isna().sum()

In [281]:
# 컬럼명 확인
# kgcon.columns
# kgsep.columns


# 컬럼명 불필요 문자 제거
kgcon.columns = kgcon.columns.str.replace('순손실', '')
kgcon.columns = kgcon.columns.str.replace('(', '')
kgcon.columns = kgcon.columns.str.replace(')', '')
kgcon.columns = kgcon.columns.str.replace('영업수익', '')
kgcon.columns = kgcon.columns.str.replace('*', '')
kgcon.columns = kgcon.columns.str.replace('연결', '')
kgcon.columns = kgcon.columns.str.replace('손실', '')
kgcon.columns = kgcon.columns.str.replace('천원', '')

kgsep.columns = kgsep.columns.str.replace('순손실', '')
kgsep.columns = kgsep.columns.str.replace('(', '')
kgsep.columns = kgsep.columns.str.replace(')', '')
kgsep.columns = kgsep.columns.str.replace('영업수익', '')
kgsep.columns = kgsep.columns.str.replace('*', '')
kgsep.columns = kgsep.columns.str.replace('연결', '')
kgsep.columns = kgsep.columns.str.replace('손실', '')
kgsep.columns = kgsep.columns.str.replace('천원', '')

#칼럼 이름 바꾸기
kgcon.rename(columns = {'영업이익':'영업손익'},inplace=True)
kgsep.rename(columns = {'영업이익':'영업손익'},inplace=True)

# 컬럼명 재정렬
kgcon = kgcon[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '자산', '자본', '영업손익', '당기순이익', '매출액']]
kgsep = kgsep[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '자산', '자본', '영업손익', '당기순이익', '매출액']]

<ipython-input-281-81a3aee03db5>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  kgcon.columns = kgcon.columns.str.replace('(', '')
<ipython-input-281-81a3aee03db5>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  kgcon.columns = kgcon.columns.str.replace(')', '')
<ipython-input-281-81a3aee03db5>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  kgcon.columns = kgcon.columns.str.replace('*', '')
<ipython-input-281-81a3aee03db5>:17: FutureWarning: The default value of regex will change from True to False in a future version

#### 연결기업 결측치를 개별기업 데이터로 대체

In [282]:
kgconsep = kgcon.copy()

# 연결기업 데이터프레임의 결측치를 개별기업 데이터로 대체
for i in kgconsep.columns:
    kgconsep[i] = kgconsep[i].fillna(kgsep[i])


# ts연결별도병합 데이터 프레임 저장
kgconsep.to_csv('데이터/1차 전처리/gaap성장성연결별도병합.csv', encoding='CP949', index=False)

In [283]:
# 연결, 별도, 병합 데이터 비교하고 확인
print('<연결>')
print('행렬 수 : ', kgcon.shape)
print('회사 수 : ', kgcon['회사명'].nunique())

print('\n<별도>')
print('행렬 수 : ', kgsep.shape)
print('회사 수 : ', kgsep['회사명'].nunique())

print('\n<병합>')
print('행렬 수 : ', kgconsep.shape)
print('회사 수 : ', kgconsep['회사명'].nunique())

<연결>
행렬 수 :  (20831, 10)
회사 수 :  2388

<별도>
행렬 수 :  (20831, 10)
회사 수 :  2388

<병합>
행렬 수 :  (20831, 10)
회사 수 :  2388


#### 파생변수 만들기
    컬럼 하나 새로 만들어서
    포문 공식 : {(당기 - 전기) / 전기} * 100 값을 채워넣으려고 함
    근데 만약 전기영업손익 < 0 면, 계산 => {(전기 - 당기) / 당기} * 100 로 채워넣을거임

In [284]:
# 전기 데이터 만들기
kgconsep['전기영업손익'] = kgconsep['영업손익'].shift().where(kgconsep.회사명.eq(kgconsep.회사명.shift()))
kgconsep['전기순이익'] = kgconsep['당기순이익'].shift().where(kgconsep.회사명.eq(kgconsep.회사명.shift()))
kgconsep['전기매출액'] = kgconsep['매출액'].shift().where(kgconsep.회사명.eq(kgconsep.회사명.shift()))
kgconsep['전기자산'] = kgconsep['자산'].shift().where(kgconsep.회사명.eq(kgconsep.회사명.shift()))
kgconsep['전기자본'] = kgconsep['자본'].shift().where(kgconsep.회사명.eq(kgconsep.회사명.shift()))


kgconsep2 = kgconsep.copy()

# 컬럼명 재정렬
kgconsep2 = kgconsep2[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '자산', '전기자산', '자본', '전기자본', '영업손익', '전기영업손익', '당기순이익', '전기순이익', '매출액', '전기매출액']]


# 성장성 재무비율 구하기
kgconsep2['총자산증가율'] = ((kgconsep2['자산'] - kgconsep2['전기자산'])/kgconsep2['전기자산'])*100
kgconsep2['자기자본증가율'] = ((kgconsep2['자본'] - kgconsep2['전기자본'])/abs(kgconsep2['전기자본']))*100
for i, row in kgconsep2.iterrows () :
    if kgconsep2.at[i,'전기영업손익'] > 0 or kgconsep2.at[i,'전기영업손익'] < 0 and kgconsep2.at[i,'영업손익'] < 0:
        kgconsep2.at[i,'정상영업손익증가율'] = ((kgconsep2.at[i,'영업손익'] - kgconsep2.at[i,'전기영업손익'])/kgconsep2.at[i,'전기영업손익'])*100
    elif kgconsep2.at[i,'전기영업손익'] < 0 and kgconsep2.at[i,'영업손익'] > 0:
        kgconsep2.at[i,'정상영업손익증가율'] = ((kgconsep2.at[i,'전기영업손익'] - kgconsep2.at[i,'영업손익'])/kgconsep2.at[i,'영업손익'])*100

for i, row in kgconsep2.iterrows () :
    if kgconsep2.at[i,'전기순이익'] > 0 or kgconsep2.at[i,'전기순이익'] < 0 and kgconsep2.at[i,'당기순이익'] < 0:
        kgconsep2.at[i,'순이익증가율'] = ((kgconsep2.at[i,'당기순이익'] - kgconsep2.at[i,'전기순이익'])/kgconsep2.at[i,'전기순이익'])*100
    elif kgconsep2.at[i,'전기순이익'] < 0 and kgconsep2.at[i,'당기순이익'] > 0:
        kgconsep2.at[i,'순이익증가율'] = ((kgconsep2.at[i,'전기순이익'] - kgconsep2.at[i,'당기순이익'])/kgconsep2.at[i,'당기순이익'])*100

for i, row in kgconsep2.iterrows () :
    if kgconsep2.at[i,'전기매출액'] > 0 or kgconsep2.at[i,'전기매출액'] < 0 and kgconsep2.at[i,'매출액'] < 0:
        kgconsep2.at[i,'매출액증가율'] = ((kgconsep2.at[i,'매출액'] - kgconsep2.at[i,'전기매출액'])/kgconsep2.at[i,'전기매출액'])*100
    elif kgconsep2.at[i,'전기매출액'] < 0 and kgconsep2.at[i,'매출액'] > 0:
        kgconsep2.at[i,'매출액증가율'] = ((kgconsep2.at[i,'전기매출액'] - kgconsep2.at[i,'매출액'])/kgconsep2.at[i,'매출액'])*100


# 데이터 프레임 저장
kgconsep2.to_csv('데이터/1차 전처리/gaap성장성파생변수.csv', encoding='CP949', index=False)

--------

## K-GAAP과 K-IFRS 합치기

In [285]:
# consep2 : K-IFRS 최종 데이터 /  kgconsep2 : K-GAAP 최종 데이터
## 파생변수들만 냅두고 위아래로 합치기
## 

---

### 자기자본구성비율, 재고자산대순운전자본비율, 자기자본회전률 파생변수 만들기

#### TS노션데이터 연결, 개별 합치기

In [286]:
# 데이터 불러오기
conno = pd.read_csv('데이터/data/연결노션정리.csv', encoding='CP949')
sepno = pd.read_csv('데이터/data/별도노션정리.csv', encoding='CP949')
defno = pd.read_csv('데이터/data/노션정리그대로.csv', encoding='CP949')

# 연결, 병합 데이터 비교하고 확인
print('<연결>')
print('행렬 수 : ', conno.shape)
print('회사 수 : ', conno['회사명'].nunique())

print('\n<개별>')
print('행렬 수 : ', sepno.shape)
print('회사 수 : ', sepno['회사명'].nunique())

print('\n<그대로>')
print('행렬 수 : ', defno.shape)
print('회사 수 : ', defno['회사명'].nunique())

<연결>
행렬 수 :  (20947, 27)
회사 수 :  2456

<개별>
행렬 수 :  (20947, 27)
회사 수 :  2456

<그대로>
행렬 수 :  (20947, 12)
회사 수 :  2456


In [287]:
# 데이터 프레임 결측치 합 확인
# conno.isna().sum()
# sepno.isna().sum()
# defno.isna().sum()

In [288]:
# 컬럼명 확인
# conno.columns
# sepno.columns


# 컬럼명 불필요 문자 제거
conno.columns = conno.columns.str.replace('수익', '')
conno.columns = conno.columns.str.replace('(', '')
conno.columns = conno.columns.str.replace(')', '')
conno.columns = conno.columns.str.replace('IFRS연결', '')
conno.columns = conno.columns.str.replace('*', '')
conno.columns = conno.columns.str.replace('보고서기재', '')
conno.columns = conno.columns.str.replace('손실', '')
conno.columns = conno.columns.str.replace('정상', '')
conno.columns = conno.columns.str.replace(' ', '')
# conno.columns

sepno.columns = sepno.columns.str.replace('수익', '')
sepno.columns = sepno.columns.str.replace('(', '')
sepno.columns = sepno.columns.str.replace(')', '')
sepno.columns = sepno.columns.str.replace('IFRS', '')
sepno.columns = sepno.columns.str.replace('*', '')
sepno.columns = sepno.columns.str.replace('보고서기재', '')
sepno.columns = sepno.columns.str.replace('손실', '')
sepno.columns = sepno.columns.str.replace('정상', '')
sepno.columns = sepno.columns.str.replace(' ', '')
# sepno.columns


# 칼럼 재배치
conno = conno[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '영업손익', '매출액', '당기순이익', '자본', '이자비용', '이자',
       '배당금', '영업활동으로인한현금흐름간접법', '발행한주식총수', '자산', '현금및현금성자산', '유동부채', '재고자산',
       '유동자산', '부채', '합계_기말인원', '법인세비용차감전계속사업손익', '인건비', '금융비용', '임차료',
       '세금과공과', '감가상각비']]

sepno = sepno[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '영업손익', '매출액', '당기순이익', '자본', '이자비용', '이자',
       '배당금', '영업활동으로인한현금흐름간접법', '발행한주식총수', '자산', '현금및현금성자산', '유동부채', '재고자산',
       '유동자산', '부채', '합계_기말인원', '법인세비용차감전계속사업손익', '인건비', '금융비용', '임차료',
       '세금과공과', '감가상각비']]

<ipython-input-288-929d5e5f5a0b>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  conno.columns = conno.columns.str.replace('(', '')
<ipython-input-288-929d5e5f5a0b>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  conno.columns = conno.columns.str.replace(')', '')
<ipython-input-288-929d5e5f5a0b>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  conno.columns = conno.columns.str.replace('*', '')
<ipython-input-288-929d5e5f5a0b>:19: FutureWarning: The default value of regex will change from True to False in a future version

#### 연결기업 결측치를 개별기업 데이터로 대체

In [289]:
consepno = conno.copy()

# 노션_연결 데이터프레임의 컬럼명을 리스트로 저장
consepno_list = consepno.columns.tolist()


# 노션_연결기업 데이터프레임의 결측치를 개별기업 데이터로 대체
for i in consepno_list:
    consepno[i] = consepno[i].fillna(sepno[i])


# ts연결별도병합 데이터 프레임 저장
consepno.to_csv('데이터/1차 전처리/노션연결별도병합.csv', encoding='CP949', index=False)

#### 노션그대로 데이터 연결병합에 합치기

In [290]:
defno2 = defno.copy()
concat = pd.merge(consepno, defno2, on = ['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일'], how='left')
concat

# 데이터 프레임 저장
concat.to_csv('데이터/1차 전처리/concat.csv', encoding='CP949', index=False)

In [291]:
# 연결, 별도, 병합, 콘켓 데이터 비교하고 확인
print('연결')
print('conno : ', conno.shape)
print('conno : ', conno['회사명'].nunique())

print('\n별도')
print('sepno : ', sepno.shape)
print('sepno : ', sepno['회사명'].nunique())

print('\n그대로')
print('defno : ', defno.shape)
print('defno : ', defno['회사명'].nunique())

print('\n병합')
print('consepno : ', consepno.shape)
print('consepno : ', consepno['회사명'].nunique())

print('\n콘켓')
print('concat : ', concat.shape)
print('concat : ', concat['회사명'].nunique())

연결
conno :  (20947, 27)
conno :  2456

별도
sepno :  (20947, 27)
sepno :  2456

그대로
defno :  (20947, 12)
defno :  2456

병합
consepno :  (20947, 27)
consepno :  2456

콘켓
concat :  (20947, 34)
concat :  2456


#### 파생변수 만들기

In [292]:
concat2 = concat.copy()

concat2['자기자본구성비율'] = (concat2['자본']/concat2['자산'])*100
concat2['재고자산대순운전자본비율'] = (concat2['재고자산']/(concat2['유동자산']-concat2['유동부채']))*100
concat2['전기자본'] = concat2['자본'].shift().where(concat2.회사명.eq(concat2.회사명.shift()))

last_cap = concat2.groupby('회사명', as_index=False)['전기자본'].nth(0).index   # 각 기업의 '전기자본'컬럼의 첫번째 행에 매출액/자본 공식 대입
concat2.iloc[last_cap, :]
concat2['자기자본회전률'] = concat2['매출액']/((concat2['전기자본'] + concat2['자본'])/2)
concat2.loc[last_cap, '자기자본회전률'] = concat2['매출액']/(concat2['자본']) 


# 데이터 프레임 저장
concat2.to_csv('데이터/1차 전처리/노션정리파생변수.csv', encoding='CP949', index=False)